### Code du professeur fonctionnel ###


In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt 

data = fits.getdata('image_test/Tarantula_Nebula-sii.fit')

plt.imshow(data , cmap='gray')
plt.colorbar()
plt.show()

In [9]:
from astroquery.mast import Observations
from astropy.coordinates import SkyCoord
from astropy import units as u
import pandas as pd
import os
import numpy as np

def get_filtered_observations(ra, dec, radius, satellite_name):
    obs = Observations.query_object(f"{ra} {dec}", radius=radius)
    return obs[obs['obs_collection'] == satellite_name]

def get_most_common_prefix(obs_ids):
    prefixes = obs_ids.str.extract('(^[a-zA-Z0-9_]+)', expand=False)
    return prefixes.value_counts().idxmax()

def get_sorted_observations(obs_filtered_by_prefix, ra, dec):
    coord = SkyCoord(ra, dec, unit=(u.deg, u.deg), frame='icrs')
    obs_coords = SkyCoord(obs_filtered_by_prefix['s_ra'], obs_filtered_by_prefix['s_dec'], unit=(u.deg, u.deg), frame='icrs')
    distances = coord.separation(obs_coords)
    obs_filtered_by_prefix['distance'] = distances
    return obs_filtered_by_prefix[np.argsort(obs_filtered_by_prefix['distance'])]

def download_fits_files(obs_filtered_by_prefix_sorted, download_directory):
    os.makedirs(download_directory, exist_ok=True)
    fits_downloaded = 0
    first_file = True
    file_sizes = []
    for obs in obs_filtered_by_prefix_sorted:
        if fits_downloaded >= 3:
            break
        product_list = Observations.get_product_list(obs)
        science_products = Observations.filter_products(product_list, productType="SCIENCE", extension="fits")
        if len(science_products) > 0:
            file_size = science_products[0]['size']
            print(f"Taille du fichier : {file_size / (1024 * 1024):.2f} MB")
            if first_file:
                user_input = input(f"Voulez-vous télécharger les fichiers, ils pèsent {file_size / (1024 * 1024):.2f} MB? (o/n): ").strip().lower()
                if user_input == 'o':
                    Observations.download_products(science_products[:3], download_dir=download_directory, cache=False)
                    fits_downloaded += 1
                    file_sizes.append(file_size)
                    first_file = False
                else:
                    print("Téléchargement annulé. Essayez un autre satellite qui aura une image moins lourde.")
                    break
            else:
                Observations.download_products(science_products[:3], download_dir=download_directory, cache=False)
                fits_downloaded += 1
                file_sizes.append(file_size)

    if len(file_sizes) == 3 and len(set(file_sizes)) == 1:
        print("Les trois fichiers téléchargés ont la même taille.")
    else:
        print("Les fichiers téléchargés n'ont pas la même taille.")


def main():
    ra = 279.23473479
    dec = 38.78368896
    radius = "0.01 deg"
    satellite_name = "JWST"
    obs_filtered = get_filtered_observations(ra, dec, radius, satellite_name)
    if len(obs_filtered) > 0:
        obs_ids = pd.Series(obs_filtered['obs_id'], dtype=str)
        most_common_prefix = get_most_common_prefix(obs_ids)
        mask = obs_ids.str.startswith(most_common_prefix).to_numpy()
        obs_filtered_by_prefix = obs_filtered[mask]
        obs_filtered_by_prefix_sorted = get_sorted_observations(obs_filtered_by_prefix, ra, dec)
        download_directory = "image_test/"
        download_fits_files(obs_filtered_by_prefix_sorted, download_directory)

if __name__ == "__main__":
    main()


Taille du fichier : 112.10 MB
Taille du fichier : 112.10 MB
Taille du fichier : 112.10 MB
Les trois fichiers téléchargés ont la même taille.
